In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=2)
today, yesterday

(datetime.date(2022, 3, 5), datetime.date(2022, 3, 3))

In [2]:
today = today - timedelta(days=1)
wkago = today - timedelta(days=7)
#wkago = date(2020, 12, 30)
today, wkago

(datetime.date(2022, 3, 4), datetime.date(2022, 2, 25))

### Restart and Run All Cells

### Weekly process or when stataus changes.

In [3]:
sqlDel = """DELETE FROM sales"""
rp = conlite.execute(sqlDel)
rp.rowcount

2725

In [5]:
sql = """
SELECT name
FROM orders 
ORDER BY name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'ASP', 'BCH', 'BDMS', 'DOHOME', 'HREIT', 'IMH', 'IP', 'JASIF', 'MEGA', 'PTT', 'RATCH', 'SINGER', 'SYNEX', 'TPIPL', 'TQM', 'TU', 'WHART'"

### Get past one year data

In [6]:
one_year_ago = yesterday - timedelta(days=365)
one_year_ago

sql = """
SELECT * 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_year_ago, in_p)
print(sql)


SELECT * 
FROM price 
WHERE date >= '2021-03-03' AND name IN ('ASP', 'BCH', 'BDMS', 'DOHOME', 'HREIT', 'IMH', 'IP', 'JASIF', 'MEGA', 'PTT', 'RATCH', 'SINGER', 'SYNEX', 'TPIPL', 'TQM', 'TU', 'WHART') 
ORDER BY name, date


In [7]:
df = pd.read_sql(sql, const)
df.shape

(4109, 7)

In [8]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name

df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)

In [10]:
sql = """
SELECT name,trade 
FROM orders 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
orders = pd.read_sql(sql, conlite)
orders.set_index(['name'],inplace=True)
orders

,trade
name,
ASP,B
BCH,S
BDMS,B
DOHOME,S
HREIT,S
IMH,B
IP,S
JASIF,S
MEGA,B


### Create monitors from stocks

In [9]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","I","O","S")
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors

,status
name,
ASP,O
BAM,O
BCH,I
DOHOME,I
GLOBAL,O
IMH,O
IP,I
IVL,I
LPF,I


In [10]:
monitors.shape[0]

22

In [11]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

orders.to_csv(data_file, header=None)
orders.to_csv(box_file)

In [12]:
sql = """
SELECT status, COUNT(*) AS items 
FROM stocks 
WHERE name IN (%s)
GROUP BY status
ORDER BY status
"""
sql = sql % in_p
grp = pd.read_sql(sql, conlite)
grp

,status,items
0,B,4
1,I,12
2,O,6


In [12]:
file_name = 'price-uploads.csv'
url = data_path + file_name
url

'../data/price-uploads.csv'

In [13]:
uploads = pd.read_csv(url)
uploads.sort_values(['name'],ascending=[True]).shape

(124, 1)

In [15]:
df_merge = pd.merge(orders, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(124, 3)

In [16]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,trade,_merge


In [17]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(107, 3)

In [18]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
MAKRO,B,SET
MEGA,B,SET100
RCL,B,SET100
STA,B,SET100
SYNEX,B,SET100
TISCO,B,SET50
TMT,B,SET
WHART,B,SET
BCH,I,SET100


In [19]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)
stocks.to_csv(one_file)

In [20]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""

buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
AMATA,O
ASIAN,O
ASK,O
ASP,O
BAM,O
BBL,O
BDMS,O
CHG,O
DCC,O


In [21]:
buy_candidates.shape[0]

32

In [22]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
BCH,I
DOHOME,I
HREIT,I
IP,I
JASIF,I
PTT,I
RATCH,I
SINGER,I


In [23]:
sell_candidates.shape[0]

8